In [1]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re


In [2]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    #username = 'gleisson.santos'
    #password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
        #'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [6]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        #'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [180]:
#df = pd.read_sql('SELECT * FROM [Hds700]', executar_conexao_bd())
df = pd.read_sql('SELECT * FROM Hds700', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS','Tipo', 'Serviço','Ano'])
#df = df.astype(str)

# Filtra as linhas onde "Tipo" é igual a "26 - MANUTENCAO DE HIDROMETRO" e "Serviço" é igual a "12 - SUBSTITUICAO HID." OU "766 - REV HIDROMETRO PARADO"
df = df.loc[(df['Tipo'] == '26 - MANUTENCAO DE HIDROMETRO') & ((df['Serviço'] == '12 - SUBSTITUICAO HID.') | (df['Serviço'] == '766 - REV HIDROMETRO PARADO'))]

# Seleciona apenas a coluna "Serviço"
df_10 = df.head(100)

display(df_10)
n_linhas = len(df_10)
print("Número de linhas: ", n_linhas)



Executado a conexão com o BD......


C:\Users\55719\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,SS,OS,Tipo,Serviço,Ano
571,950327746,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
575,950331729,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
578,951743416,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
580,953725963,1,26 - MANUTENCAO DE HIDROMETRO,766 - REV HIDROMETRO PARADO,2020.0
581,953725964,1,26 - MANUTENCAO DE HIDROMETRO,766 - REV HIDROMETRO PARADO,2020.0
...,...,...,...,...,...
713,951957330,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
716,953987467,2,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
717,953987468,2,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0
721,954009408,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020.0


Número de linhas:  100


In [195]:
#Inicinado automação / abrindo navegador
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\55719\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\55719\AppData\Local\Temp\ipykernel_2136\4068665143.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [196]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [197]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(2)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
#arvore_acesso(driver, 'insum', 'GOSI_anchor')

In [198]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').clear()
    
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-panel"]/input').click()
    
preencher_ss(driver, '953725963', '1')

In [199]:
um_meio = ['INTDIG01 - HIDROMETRO RES MONO 0,75M3/H 1/2"S/C 45',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 1,5 M3/H',           
           'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

de_tres = ['INTDIG01 - HIDROMETRO RES MULT 1,5M3/H 1/2" S/C 45º',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 3 E 1,5 M3/H',           
          'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

In [173]:
campo = driver.find_element(By.XPATH, '//*[@id="form-j_idt135-j_idt147-item"]')
texto = campo.text
print(texto)


HABITADA LIGADO HD A15S391566 LT 0319 FIZEMOS REVISAO NO HD ESTA NORMAL


In [200]:
def campo_material(driver, material1, material2, material3):
    time.sleep(1)
    campo = driver.find_element(By.XPATH, '//*[@id="form-j_idt135-j_idt147-item"]')
    texto = campo.text
    
    padrao_ay = r"[AY][0-9]+"
    resultados = re.findall(padrao_ay, texto)
    
    try:
        campo1 = driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')
    except NoSuchElementException:
        print("Erro na Unidade de medição")
        return
    
    try:
        if any(resultado.startswith('Y') for resultado in resultados):
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(0.5)
            campo1.send_keys(material1)
            time.sleep(2)
            campo1.send_keys(Keys.RETURN)
            time.sleep(1)

        elif any(resultado.startswith('A') for resultado in resultados):
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(0.5)
            campo1.send_keys(material2)
            time.sleep(2)
            campo1.send_keys(Keys.RETURN)
            time.sleep(1)

        else:
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(0.5)
            campo1.send_keys(material3)
            time.sleep(2)
            campo1.send_keys(Keys.RETURN)
            time.sleep(1)
    except NoSuchElementException:
        print("Erro ao preencher campo de material")


In [201]:
campo_material(driver, um_meio, de_tres, um_meio)

In [138]:
#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir):
    campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
    campo2.click()
    campo2.clear()
    campo2.send_keys(qtd)
    time.sleep(0.5)
    #Acessando e inserindo Disp p medir S/N
    select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
    select = Select(select_element)
    select.select_by_value(medir)
    # select.select_by_visible_text('Não')
    time.sleep(0.5)
    #Confirar insumo
    driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()
    
#campo_quantidade(driver, '1', 'N')

In [ ]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    comando_sql = "UPDATE [Insumar] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [ ]:
def geral():
    #Inicinado automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(1)
    
    login = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(1)
      

    # Procurar a tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(1)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    #itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - CAP PVC JS 25MM']

    
    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for item in itens:
            campo_material(driver, item)            
            time.sleep(1)
            campo_quantidade(driver, '3', 'N')
            time.sleep(1)
            print("Materiais insumados com sucesso! :)")
   
    

In [28]:
def esperar_sumir(driver):
    wait = WebDriverWait(driver, 30)
    element = wait.until(EC.invisibility_of_element_located((By.ID, "j_idt24_modal")))

In [ ]:
geral()

In [202]:
um_meio = ['INTDIG01 - HIDROMETRO RES MONO 0,75M3/H 1/2"S/C 45',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 1,5 M3/H',           
           'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

de_tres = ['INTDIG01 - HIDROMETRO RES MULT 1,5M3/H 1/2" S/C 45º',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 3 E 1,5 M3/H',           
          'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']


def esperar_sumir(driver):
    wait = WebDriverWait(driver, 30)
    element = wait.until(EC.invisibility_of_element_located((By.ID, "j_idt24_modal")))

def login(driver, user, senha):
    
    # Esperar pela tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(user)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(0.5)

def geral2():
    #Iniciando automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(0.5)
    
    user = 't034183'
    senha = 'CNB@2022'
    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(0.5)
    
    # Esperar pela tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(user)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(0.5)
    
    arvore_acesso(driver, 'insum', 'GOSI_anchor')
    time.sleep(1)
    
    contador = 0

    for i, row in df_10.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        for material1, material2, material1 in zip(um_meio, de_tres, um_meio):
            campo_material(driver, material1, material2, material1)
            time.sleep(0.5)
            esperar_sumir(driver)
            campo_quantidade(driver, 1, 'N')
            time.sleep(0.5)
            contador += 1

    print(f"{contador} Ss(s) insumada(s) com sucesso!")



In [203]:
geral2()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\55719\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\55719\AppData\Local\Temp\ipykernel_2136\3804491754.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.66)
Stacktrace:
Backtrace:
	(No symbol) [0x002FDCE3]
	(No symbol) [0x002939D1]
	(No symbol) [0x001A4DA8]
	(No symbol) [0x0018D0D3]
	(No symbol) [0x001EEA8B]
	(No symbol) [0x001FD093]
	(No symbol) [0x001EACC6]
	(No symbol) [0x001C6F68]
	(No symbol) [0x001C80CD]
	GetHandleVerifier [0x00573832+2506274]
	GetHandleVerifier [0x005A9794+2727300]
	GetHandleVerifier [0x005AE36C+2746716]
	GetHandleVerifier [0x003A6690+617600]
	(No symbol) [0x0029C712]
	(No symbol) [0x002A1FF8]
	(No symbol) [0x002A20DB]
	(No symbol) [0x002AC63B]
	BaseThreadInitThunk [0x759800F9+25]
	RtlGetAppContainerNamedObjectPath [0x77417BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77417B8E+238]
	(No symbol) [0x00000000]
